### Importing the libraries

In [1]:
import pandas as pd
import numpy as np
import math
import warnings
warnings.filterwarnings("ignore")

#### Problem Statement

Q. What to do?
- verify if the charges levied by their partners per Order are correct


Q. How is the courier price calculated?
- Weight of the product
- Distance between the warehouse and customer’s delivery address


1. LHS data by client

- Order ID is common identifier
- compare wights from both the datas
- weight slabs of 0.5 KG multiples
- compare distance by delievry zones

2. RHS data by courier company

- fwd charges / fwd charges+RTO tax

### Importing the data

In [2]:
order_report_x=pd.read_excel(r'C:\Users\Administrator\OneDrive\Desktop\Data Science\Project\Cointab\Company X - Order Report.xlsx')
pincode_zone_x=pd.read_excel(r'C:\Users\Administrator\OneDrive\Desktop\Data Science\Project\Cointab\Company X - Pincode Zones.xlsx')
SKU_Master_x=pd.read_excel(r'C:\Users\Administrator\OneDrive\Desktop\Data Science\Project\Cointab\Company X - SKU Master.xlsx')
Courier_Invoice=pd.read_excel(r'C:\Users\Administrator\OneDrive\Desktop\Data Science\Project\Cointab\Courier Company - Invoice.xlsx')
Courier_Rates=pd.read_excel(r'C:\Users\Administrator\OneDrive\Desktop\Data Science\Project\Cointab\Courier Company - Rates.xlsx')

#### Data preview for understanding the data

#### LHS Data (X-Company)

In [3]:
order_report_x.head(3)

,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0


In [4]:
pincode_zone_x.head(3)

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d


In [5]:
SKU_Master_x.head(3)

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113


#### RHS data (Courier Company)

In [6]:
Courier_Invoice.head(3)

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0
1,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2
2,1091117222931,2001806408,2.5,121003,532484,d,Forward charges,224.6


In [7]:
Courier_Rates

,fwd_a_fixed,fwd_a_additional,fwd_b_fixed,fwd_b_additional,fwd_c_fixed,fwd_c_additional,fwd_d_fixed,fwd_d_additional,fwd_e_fixed,fwd_e_additional,rto_a_fixed,rto_a_additional,rto_b_fixed,rto_b_additional,rto_c_fixed,rto_c_additional,rto_d_fixed,rto_d_additional,rto_e_fixed,rto_e_additional
0,29.5,23.6,33,28.3,40.1,38.9,45.4,44.8,56.6,55.5,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5


### Merging the LHS data

#### Joining the data of X-Company

- to compare the data with the Courier company data
- as orderID is common variable between both data, we have to merge the X-company's data first to be able to compare


In [8]:
X_data=pd.merge(order_report_x,SKU_Master_x,left_on=["SKU"],right_on=["SKU"],how="inner")
rename_order_id=X_data.rename(columns={"ExternOrderNo":"Order ID"},inplace=True)
X_data

,Order ID,SKU,Order Qty,Weight (g)
0,2001827036,8904223818706,1.0,127
1,2001821995,8904223818706,1.0,127
2,2001819252,8904223818706,1.0,127
3,2001816996,8904223818706,1.0,127
4,2001814580,8904223818706,1.0,127
...,...,...,...,...
396,2001806616,8904223819123,1.0,250
397,2001806567,8904223815804,1.0,160
398,2001806567,8904223818577,1.0,150
399,2001806408,8904223819437,2.0,552


#### Linking pincode data to OrderID

In [9]:
pincode_zone_x = pincode_zone_x.drop_duplicates(subset=['Customer Pincode'])

In [10]:
Courier_Invoice_copy=Courier_Invoice[['Order ID', 'Customer Pincode','Type of Shipment']]

In [11]:
pincode_data=Courier_Invoice_copy.merge(pincode_zone_x,on='Customer Pincode')

In [12]:
pincode_data

,Order ID,Customer Pincode,Type of Shipment,Warehouse Pincode,Zone
0,2001806232,507101,Forward charges,121003,d
1,2001806273,486886,Forward charges,121003,d
2,2001806408,532484,Forward charges,121003,d
3,2001806458,143001,Forward charges,121003,b
4,2001807012,515591,Forward charges,121003,d
...,...,...,...,...,...
119,2001811039,302020,Forward charges,121003,b
120,2001811305,302020,Forward charges,121003,b
121,2001812941,325207,Forward charges,121003,b
122,2001809383,303702,Forward and RTO charges,121003,b


In [13]:
order_x=X_data.merge(pincode_data,on="Order ID")
order_x

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Zone
0,2001827036,8904223818706,1.0,127,173213,Forward charges,121003,e
1,2001827036,8904223819093,1.0,150,173213,Forward charges,121003,e
2,2001827036,8904223819109,1.0,100,173213,Forward charges,121003,e
3,2001827036,8904223818430,1.0,165,173213,Forward charges,121003,e
4,2001827036,8904223819277,1.0,350,173213,Forward charges,121003,e
...,...,...,...,...,...,...,...,...
396,2001806251,8904223819161,1.0,115,743263,Forward charges,121003,d
397,2001806251,8904223819260,1.0,130,743263,Forward charges,121003,d
398,2001806233,8904223819161,1.0,115,263139,Forward charges,121003,b
399,2001806233,8904223819260,1.0,130,263139,Forward charges,121003,b


#### Creating weight slab function so that we can calculate the charges

In [14]:
def round_of(x):
    y=round(x)
    if ((x-y)>0):
        return(y+0.5)
    else:
        return(y)

In [15]:
def weight_Slab(n):
    for i in range(len(n)):
        n[i]=round_of(n[i])

### Creating the table of comparison

#### For Courier company data

In [16]:
Output1=pd.DataFrame(columns=["Order ID","AWB Code","Total weight as per Courier Company (KG)","Weight slab charged by Courier Company (KG)","Delivery Zone charged by Courier Company","Charges Billed by Courier Company (Rs.)","Difference Between Expected Charges and Billed Charges (Rs.)"])

- Importing the values into the data

In [17]:
Output1["Order ID"]=Courier_Invoice["Order ID"]
Output1["AWB Code"]=Courier_Invoice["AWB Code"]
Output1["Total weight as per Courier Company (KG)"]=Courier_Invoice["Charged Weight"]
Output1["Delivery Zone charged by Courier Company"]=Courier_Invoice["Zone"]
Output1["Charges Billed by Courier Company (Rs.)"]=Courier_Invoice["Billing Amount (Rs.)"]
Output1["Weight slab charged by Courier Company (KG)"]=Courier_Invoice["Charged Weight"]
weight_Slab(Output1["Weight slab charged by Courier Company (KG)"])

#### For X company data

In [18]:
Output2=pd.DataFrame(columns=["Order ID","Total weight as per X (KG)","Weight slab as per X (KG)","Delivery Zone as per X","Type of Shipment",'Expected Charge as per X (Rs.)'])

- Importing the values into the data

In [19]:
Output2["Order ID"]=order_x["Order ID"]
Output2["Total weight as per X (KG)"]=order_x["Weight (g)"]/1000
Output2["Delivery Zone as per X"]=order_x["Zone"]
Output2["Weight slab as per X (KG)"]=order_x["Weight (g)"]/1000
weight_Slab(Output2["Weight slab as per X (KG)"])
Output2["Type of Shipment"]=order_x["Type of Shipment"]

#### Here, we are calculating the Expected charges based on delievery zone and weight slab of the order

In [20]:

for i in range(len(Output2)):
    if (Output2.iloc[i][3]=="a" and Output2.iloc[i][4]== "Forward charges"):
        Output2["Expected Charge as per X (Rs.)"][i]= float(Courier_Rates["fwd_a_fixed"]+((Output2["Weight slab as per X (KG)"][i]/0.5)-1)*Courier_Rates["fwd_a_additional"])
    if (Output2.iloc[i][3]=="b" and Output2.iloc[i][4]== "Forward charges"):
        Output2["Expected Charge as per X (Rs.)"][i]= float(Courier_Rates["fwd_b_fixed"]+((Output2["Weight slab as per X (KG)"][i]/0.5)-1)*Courier_Rates["fwd_b_additional"])
    if (Output2.iloc[i][3]=="c" and Output2.iloc[i][4]== "Forward charges"):
        Output2["Expected Charge as per X (Rs.)"][i]= float(Courier_Rates["fwd_c_fixed"]+((Output2["Weight slab as per X (KG)"][i]/0.5)-1)*Courier_Rates["fwd_c_additional"])
    if (Output2.iloc[i][3]=="d" and Output2.iloc[i][4]== "Forward charges"):
        Output2["Expected Charge as per X (Rs.)"][i]= float(Courier_Rates["fwd_d_fixed"]+((Output2["Weight slab as per X (KG)"][i]/0.5)-1)*Courier_Rates["fwd_d_additional"])
    if (Output2.iloc[i][3]=="e" and Output2.iloc[i][4]== "Forward charges"):
        Output2["Expected Charge as per X (Rs.)"][i]= float(Courier_Rates["fwd_e_fixed"]+((Output2["Weight slab as per X (KG)"][i]/0.5)-1)*Courier_Rates["fwd_e_additional"])
    if (Output2.iloc[i][3]=="a" and Output2.iloc[i][4]== "Forward and RTO charges"):
        Output2["Expected Charge as per X (Rs.)"][i]= float(Courier_Rates["fwd_a_fixed"]+((Output2["Weight slab as per X (KG)"][i]/0.5)-1)*Courier_Rates["rto_a_additional"]+Courier_Rates["rto_a_fixed"])
    if (Output2.iloc[i][3]=="b" and Output2.iloc[i][4]== "Forward and RTO charges"):
        Output2["Expected Charge as per X (Rs.)"][i]= float(Courier_Rates["fwd_b_fixed"]+((Output2["Weight slab as per X (KG)"][i]/0.5)-1)*Courier_Rates["rto_b_additional"]+Courier_Rates["rto_b_fixed"])
    if (Output2.iloc[i][3]=="c" and Output2.iloc[i][4]== "Forward and RTO charges"):
        Output2["Expected Charge as per X (Rs.)"][i]= float(Courier_Rates["fwd_c_fixed"]+((Output2["Weight slab as per X (KG)"][i]/0.5)-1)*Courier_Rates["rto_c_additional"]+Courier_Rates["rto_c_fixed"])
    if (Output2.iloc[i][3]=="d" and Output2.iloc[i][4]== "Forward and RTO charges"):
        Output2["Expected Charge as per X (Rs.)"][i]= float(Courier_Rates["fwd_d_fixed"]+((Output2["Weight slab as per X (KG)"][i]/0.5)-1)*Courier_Rates["rto_d_additional"]+Courier_Rates["rto_d_fixed"])
    if (Output2.iloc[i][3]=="e" and Output2.iloc[i][4]== "Forward and RTO charges"):
        Output2["Expected Charge as per X (Rs.)"][i]= float(Courier_Rates["fwd_e_fixed"]+((Output2["Weight slab as per X (KG)"][i]/0.5)-1)*Courier_Rates["rto_e_additional"]+Courier_Rates["rto_e_fixed"])  

### Final Output

- In "Difference Between Expected Charges and Billed Charges (Rs.)" column :-
        - negative values indicates to be undercharged by the courier company
        - while positive values indicates to be overcharged by the courier company
        - zero indicates correctly charged

#### Creating a final table where both datas will be merged to compare

In [21]:
Output=Output2.merge(Output1,on="Order ID")

In [22]:
Output["Difference Between Expected Charges and Billed Charges (Rs.)"]=Output["Charges Billed by Courier Company (Rs.)"]-Output["Expected Charge as per X (Rs.)"]

In [23]:
Output.head()

,Order ID,Total weight as per X (KG),Weight slab as per X (KG),Delivery Zone as per X,Type of Shipment,Expected Charge as per X (Rs.),AWB Code,Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone charged by Courier Company,Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001827036,0.127,0.5,e,Forward charges,56.6,1091122418320,1.6,2.0,b,117.9,61.3
1,2001827036,0.150,0.5,e,Forward charges,56.6,1091122418320,1.6,2.0,b,117.9,61.3
2,2001827036,0.100,0.5,e,Forward charges,56.6,1091122418320,1.6,2.0,b,117.9,61.3
3,2001827036,0.165,0.5,e,Forward charges,56.6,1091122418320,1.6,2.0,b,117.9,61.3
4,2001827036,0.350,0.5,e,Forward charges,56.6,1091122418320,1.6,2.0,b,117.9,61.3


## Finding if the orders have been charged correctly or not

#### Correctly charged orders

In [24]:
correctly_charged=Output[Output["Difference Between Expected Charges and Billed Charges (Rs.)"]==0.0]
correctly_charged.head()

,Order ID,Total weight as per X (KG),Weight slab as per X (KG),Delivery Zone as per X,Type of Shipment,Expected Charge as per X (Rs.),AWB Code,Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone charged by Courier Company,Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
129,2001807290,0.127,0.5,d,Forward charges,45.4,1091117324206,0.5,0.5,d,45.4,0.0
130,2001807290,0.240,0.5,d,Forward charges,45.4,1091117324206,0.5,0.5,d,45.4,0.0
131,2001807290,0.133,0.5,d,Forward charges,45.4,1091117324206,0.5,0.5,d,45.4,0.0
135,2001807186,0.127,0.5,d,Forward charges,45.4,1091117323812,0.5,0.5,d,45.4,0.0
136,2001807186,0.240,0.5,d,Forward charges,45.4,1091117323812,0.5,0.5,d,45.4,0.0


In [25]:
Total_correctly_charged=sum(correctly_charged["Expected Charge as per X (Rs.)"])
Total_correctly_charged

507.5999999999999

In [26]:
len(correctly_charged)

12

#### Overcharged Orders

In [27]:
overcharged=Output[Output["Difference Between Expected Charges and Billed Charges (Rs.)"]>0.0]
overcharged.head()

,Order ID,Total weight as per X (KG),Weight slab as per X (KG),Delivery Zone as per X,Type of Shipment,Expected Charge as per X (Rs.),AWB Code,Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone charged by Courier Company,Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001827036,0.127,0.5,e,Forward charges,56.6,1091122418320,1.6,2.0,b,117.9,61.3
1,2001827036,0.150,0.5,e,Forward charges,56.6,1091122418320,1.6,2.0,b,117.9,61.3
2,2001827036,0.100,0.5,e,Forward charges,56.6,1091122418320,1.6,2.0,b,117.9,61.3
3,2001827036,0.165,0.5,e,Forward charges,56.6,1091122418320,1.6,2.0,b,117.9,61.3
4,2001827036,0.350,0.5,e,Forward charges,56.6,1091122418320,1.6,2.0,b,117.9,61.3


In [28]:
len(overcharged)

380

#### Undercharged Orders

In [29]:
undercharged=Output[Output["Difference Between Expected Charges and Billed Charges (Rs.)"]<0.0]
undercharged.head()

,Order ID,Total weight as per X (KG),Weight slab as per X (KG),Delivery Zone as per X,Type of Shipment,Expected Charge as per X (Rs.),AWB Code,Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone charged by Courier Company,Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
11,2001819252,0.127,0.5,e,Forward charges,56.6,1091120352712,0.3,0.5,b,33.0,-23.6
12,2001819252,0.010,0.5,e,Forward charges,56.6,1091120352712,0.3,0.5,b,33.0,-23.6
13,2001819252,0.133,0.5,e,Forward charges,56.6,1091120352712,0.3,0.5,b,33.0,-23.6
32,2001811475,0.127,0.5,e,Forward charges,56.6,1091121844806,0.5,0.5,b,33.0,-23.6
33,2001811475,0.240,0.5,e,Forward charges,56.6,1091121844806,0.5,0.5,b,33.0,-23.6


In [30]:
len(undercharged)

9

#### Total Overcharged Amount

In [31]:
Total_Overcharged=np.absolute(sum(overcharged["Difference Between Expected Charges and Billed Charges (Rs.)"]))
Total_Overcharged

32081.69999999992

#### Total Undercharged Amount

In [32]:
Total_Undercharged=np.absolute(sum(undercharged["Difference Between Expected Charges and Billed Charges (Rs.)"]))
Total_Undercharged

165.2

## Exporting the data in Excel format

### Final Output

In [33]:
data = {'Count': [len(correctly_charged),len(overcharged),len(undercharged)],
        'Amount (Rs.)': [Total_correctly_charged,Total_Overcharged,Total_Undercharged]}
  
# Creates pandas DataFrame.
Summary = pd.DataFrame(data, index=['Total orders where X has been correctly charged ',
                               'Total Orders where X has been overcharged ',
                               'Total Orders where X has been undercharged'])

### Summary

In [34]:
Summary

,Count,Amount (Rs.)
Total orders where X has been correctly charged,12,507.6
Total Orders where X has been overcharged,380,32081.7
Total Orders where X has been undercharged,9,165.2


In [35]:
with pd.ExcelWriter("C:\\Users\Administrator\OneDrive\Desktop\Data Science\Project\Cointab\Cointab.xlsx") as writer:
    Summary.to_excel(writer, sheet_name="Summary", index=True)
    Output.to_excel(writer, sheet_name="Output", index=False)